In [4]:
import numpy as np
import pickle as pkl
import os

from graph_tool import load_graph
from graph_tool.generation import lattice
from graph_tool.spectral import adjacency
from graph_tool import load_graph

from scipy.io import savemat
from glob import glob
from tqdm import tqdm

from itertools import product

from cascade_generator import si, observe_cascade
from helpers import infected_nodes, cascade_source, prepare_cascade_for_netfill


In [18]:
# graph_names = ['infectious', 'lattice-1024']
graph_names = ['grqc']
infection_proba = 0.1
# obs_fractions = [0.5, 0.6, 0.7, 0.8, 0.9]
obs_fractions = [0.5]
# cascade_fracions = [0.05]
# cascade_fracions = [0.1, 0.2, 0.3, 0.4, 0.5]
cascade_fracions = [0.05, 0.1, 0.15, 0.2, 0.25]

In [21]:
def do(graph_name, obs_fraction, cascade_fracion):
    print(graph_name, obs_fraction, cascade_fracion)
    g = load_graph('data/{}/graph_weighted_ud_{}.gt'.format(graph_name, infection_proba))
    input_dir = 'cascade/{}-msi-s{}-o{}-omuniform/'.format(
        graph_name, cascade_fracion, obs_fraction)
    output_dir = 'netfill-data/cascade/{}-msi-s{}-o{}-omuniform/'.format(
        graph_name, cascade_fracion, obs_fraction)

    for input_path in tqdm(glob(input_dir + '*.pkl')):
        output_path = os.path.join(output_dir, os.path.basename(input_path))
        prepare_cascade_for_netfill(g,
                                    beta=0.1, p=0.5,
                                    input_path=input_path,
                                    output_path=output_path)

In [22]:
for gn, of, cf in product(graph_names, obs_fractions, cascade_fracions):
    do(gn, of, cf)

 57%|█████▋    | 57/100 [00:00<00:00, 566.57it/s]

grqc 0.5 0.05


 57%|█████▋    | 57/100 [00:00<00:00, 380.94it/s]

grqc 0.5 0.1


  0%|          | 0/100 [00:00<?, ?it/s]

grqc 0.5 0.15
grqc 0.5 0.2


 48%|████▊     | 48/100 [00:00<00:00, 479.13it/s]

grqc 0.5 0.25


100%|██████████| 100/100 [00:00<00:00, 247.48it/s]


In [68]:
G = adjacency(g).todense()
savemat('netfill-data/graphs/{}.mat'.format(graph_name),
        {'G': G},
        oned_as='row',
        do_compression=True)